In [2]:
import os
import cv2
from glob import glob
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import random
import time
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
import cv2
import shutil
from sklearn.neighbors import KNeighborsClassifier
import joblib  
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

Preprocessing Step

In [4]:
# Define the input and output directories
input_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\Assignment2\isolated_words_per_user"
output_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\preprocessed_images"

# Target image size for resizing
target_size = (128, 128) 

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Preprocessing function
def preprocess_image(image_path, output_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the image
    resized_image = cv2.resize(gray_image, target_size)

    # Apply Gaussian blur for noise removal
    preprocessed_image = cv2.GaussianBlur(resized_image, (5, 5), 0)

    # Save the preprocessed image
    cv2.imwrite(output_path, preprocessed_image)

# Preprocess all images
for file_path in glob(f"{input_dir}/**/*.png", recursive=True):  # Adjust the extension if needed
    # Create the corresponding output path
    relative_path = os.path.relpath(file_path, input_dir)
    output_path = os.path.join(output_dir, relative_path)
    
    # Ensure the output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Preprocess and save the image
    preprocess_image(file_path, output_path)
    print("Preprocessing complete!")


Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing complete!
Preprocessing co

Dataset Splitting

In [5]:
# Define dataset and output directories
dataset_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\preprocessed_images"
output_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\AHAWP_splitting_to_train_and_test"

# Define dataset splits
splits = ["train", "test"]

# Create folder structure
for split in splits:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Debug: Check if dataset directory exists
if not os.path.exists(dataset_dir):
    raise FileNotFoundError(f"Dataset directory not found: {dataset_dir}")
else:
    print("Dataset directory found:", dataset_dir)

# Find all writer folders
writer_folders = [os.path.join(dataset_dir, d) for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
print(f"Found {len(writer_folders)} writer folders.")  # Debug

# Process each writer folder
for writer_folder in writer_folders:
    writer_name = os.path.basename(writer_folder)  # Get the writer's name
    image_files = glob(f"{writer_folder}/*.png")  # Find all image files in the writer's folder

    # Shuffle images to randomize splitting
    random.shuffle(image_files)

    # Split images into 80% train and 20% test
    train_split = int(0.8 * len(image_files))
    train_files = image_files[:train_split]
    test_files = image_files[train_split:]

    # Copy train files
    train_writer_dir = os.path.join(output_dir, "train", writer_name)
    os.makedirs(train_writer_dir, exist_ok=True)
    for file in train_files:
        shutil.copy(file, train_writer_dir)

    # Copy test files
    test_writer_dir = os.path.join(output_dir, "test", writer_name)
    os.makedirs(test_writer_dir, exist_ok=True)
    for file in test_files:
        shutil.copy(file, test_writer_dir)

print("Dataset organization complete: 80% training, 20% testing!")

Dataset directory found: D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\preprocessed_images
Found 82 writer folders.
Dataset organization complete: 80% training, 20% testing!


# 1. KNN Model

# 1.1. ORB Algorithm 

In [22]:
train_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\orbKeypoints_split_new\train"
test_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\orbKeypoints_split_new\test"
kmeans_model_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\kmeans_model1.pkl"
knn_model_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\knn_model1.pkl"

step_start = time.time()

k1 = 50  # Number of clusters (visual words)
n_neighbors = 20  # Number of neighbors for KNN
orb = cv2.ORB_create()

def extract_descriptors_from_saved(image_paths):
    descriptors_list = []
    labels = []

    for image_path in image_paths:
        # Locate the corresponding descriptor file
        desc_file_path = os.path.splitext(image_path)[0] + "_desc.npy"
        if os.path.exists(desc_file_path):
            descriptors = np.load(desc_file_path)  # Load saved descriptors
            descriptors_list.append(descriptors)

            # Extract label from the folder structure
            label = os.path.basename(os.path.dirname(image_path))
            labels.append(label)
        else:
            print(f"Descriptor file not found for {image_path}")
    return descriptors_list, labels

# -------------------- STEP 1: TRAIN KMEANS (BOVW) --------------------
# Get all training image paths
train_image_paths = glob(f"{train_dir}/**/*.png", recursive=True)
print(f"Found {len(train_image_paths)} training images.")

# Extract descriptors and labels
train_descriptors_list, train_labels = extract_descriptors_from_saved(train_image_paths)
all_descriptors = np.vstack(train_descriptors_list)
print(f"Total descriptors shape: {all_descriptors.shape}")

# Train KMeans
print("Training KMeans to create visual words...")
kmeans = KMeans(n_clusters=k1, random_state=42)
kmeans.fit(all_descriptors)

# Save the trained KMeans model
joblib.dump(kmeans, kmeans_model_path)
print("KMeans model saved.")

# Generate histograms for training set
X_train = [generate_histogram(des, kmeans) for des in train_descriptors_list]
Y_train = np.array(train_labels)

print(f"Feature matrix shape: {np.array(X_train).shape}, Labels shape: {Y_train.shape}")

# -------------------- STEP 2: TRAIN KNN CLASSIFIER --------------------
print("Training KNN classifier...")
knn = KNeighborsClassifier(n_neighbors=n_neighbors)
knn.fit(X_train, Y_train)

# Save the trained KNN model
joblib.dump(knn, knn_model_path)
print("KNN model saved.")

# -------------------- STEP 3: TEST AND EVALUATE --------------------
# Load the trained KMeans and KNN models
print("Loading trained models...")
kmeans = joblib.load(kmeans_model_path)
knn = joblib.load(knn_model_path)

# Get all test image paths
test_image_paths = glob(f"{test_dir}/**/*.png", recursive=True)
print(f"Found {len(test_image_paths)} test images.")

# Extract descriptors and labels for the test set
test_descriptors_list, test_labels = extract_descriptors_from_saved(test_image_paths)

# Generate histograms for test set
X_test = []
for descriptors in test_descriptors_list:
    if descriptors is not None:
        histogram = generate_histogram(descriptors, kmeans)
        X_test.append(histogram)
X_test = np.array(X_test)

print(f"Test feature matrix shape: {X_test.shape}")

# Predict test labels using the trained KNN model
print("Predicting test labels...")
predictions = knn.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Test Set Accuracy: {accuracy:.2f}")
print(f"Total time for entire process: {time.time() - start_time:.2f} seconds")


for i in range(len(test_image_paths)):
    print(f"Image: {os.path.basename(test_image_paths[i])}, True Label: {test_labels[i]}, Predicted: {predictions[i]}")

Found 6515 training images.
Total descriptors shape: (375596, 32)
Training KMeans to create visual words...
KMeans model saved.
Feature matrix shape: (6515, 50), Labels shape: (6515,)
Training KNN classifier...
KNN model saved.
Loading trained models...
Found 1629 test images.
Test feature matrix shape: (1629, 50)
Predicting test labels...
Test Set Accuracy: 0.06
Total time for entire process: 5710.23 seconds
Image: user001_abjadiyah_035.png, True Label: user001, Predicted: user017
Image: user001_fasayakfeekahum_044.png, True Label: user001, Predicted: user026
Image: user001_fasayakfeekahum_046.png, True Label: user001, Predicted: user020
Image: user001_fasayakfeekahum_050.png, True Label: user001, Predicted: user001
Image: user001_ghaleez_013.png, True Label: user001, Predicted: user052
Image: user001_ghazaal_002.png, True Label: user001, Predicted: user001
Image: user001_ghazaal_007.png, True Label: user001, Predicted: user001
Image: user001_mehras_041.png, True Label: user001, Predi

# 1.2. SIFT Algorithm

In [21]:

#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
train_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\siftKeypointsWithEvaluation_split\train"
test_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\siftKeypointsWithEvaluation_split\test"
kmeans_model_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\kmeans_modelSIFT.pkl"
knn_model_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\knn_modelSIFT.pkl"

step_start = time.time()

k2 = 55  # Number of clusters (visual words)
n_neighbors = 20  # Number of neighbors for KNN
sift = cv2.SIFT_create()

    
def extract_descriptors_from_saved(image_paths):
    descriptors_list = []
    labels = []

    for image_path in image_paths:
        # Locate the corresponding descriptor file
        desc_file_path = os.path.splitext(image_path)[0] + "_descSIFT.npy"
        if os.path.exists(desc_file_path):
            descriptors = np.load(desc_file_path)  # Load saved descriptors
            descriptors_list.append(descriptors)

            # Extract label from the folder structure
            label = os.path.basename(os.path.dirname(image_path))
            labels.append(label)
        else:
            print(f"Descriptor file not found for {image_path}")
    return descriptors_list, labels

# -------------------- STEP 1: TRAIN KMEANS (BOVW) --------------------
# Get all training image paths
train_image_paths = glob(f"{train_dir}/**/*.png", recursive=True)
print(f"Found {len(train_image_paths)} training images.")

# Extract descriptors and labels
train_descriptors_list, train_labels = extract_descriptors_from_saved(train_image_paths)
all_descriptors = np.vstack(train_descriptors_list)
print(f"Total descriptors shape: {all_descriptors.shape}")

# Train KMeans
print("Training KMeans to create visual words...")
kmeans = KMeans(n_clusters=k2, random_state=42)
kmeans.fit(all_descriptors)

# Save the trained KMeans model
joblib.dump(kmeans, kmeans_model_path)
print("KMeans model saved.")

# Generate histograms for training set
X_train = [generate_histogram(des, kmeans) for des in train_descriptors_list]
Y_train = np.array(train_labels)

print(f"Feature matrix shape: {np.array(X_train).shape}, Labels shape: {Y_train.shape}")

# -------------------- STEP 2: TRAIN KNN CLASSIFIER --------------------
print("Training KNN classifier...")
knn = KNeighborsClassifier(n_neighbors=n_neighbors)
knn.fit(X_train, Y_train)

# Save the trained KNN model
joblib.dump(knn, knn_model_path)
print("KNN model saved.")

# -------------------- STEP 3: TEST AND EVALUATE --------------------
# Load the trained KMeans and KNN models
print("Loading trained models...")
kmeans = joblib.load(kmeans_model_path)
knn = joblib.load(knn_model_path)

# Get all test image paths
test_image_paths = glob(f"{test_dir}/**/*.png", recursive=True)
print(f"Found {len(test_image_paths)} test images.")

# Extract descriptors and labels for the test set
test_descriptors_list, test_labels = extract_descriptors_from_saved(test_image_paths)

# Generate histograms for test set
X_test = []
for descriptors in test_descriptors_list:
    if descriptors is not None:
        histogram = generate_histogram(descriptors, kmeans)
        X_test.append(histogram)
X_test = np.array(X_test)

print(f"Test feature matrix shape: {X_test.shape}")

# Predict test labels using the trained KNN model
print("Predicting test labels...")
predictions = knn.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Test Set Accuracy: {accuracy:.2f}")
print(f"Total time for entire process: {time.time() - start_time:.2f} seconds")

# Print Precision, Recall, and F1-Score for each class
#precision = precision_score(test_labels, predictions, average='weighted')
#recall = recall_score(test_labels, predictions, average='weighted')
#f1 = f1_score(test_labels, predictions, average='weighted')

#print(f"Precision: {precision:.2f}")
#print(f"Recall: {recall:.2f}")
#print(f"F1-Score: {f1:.2f}")

for i in range(len(test_image_paths)):
    print(f"Image: {os.path.basename(test_image_paths[i])}, True Label: {test_labels[i]}, Predicted: {predictions[i]}")

Found 6515 training images.
Total descriptors shape: (333163, 128)
Training KMeans to create visual words...
KMeans model saved.
Feature matrix shape: (6515, 55), Labels shape: (6515,)
Training KNN classifier...
KNN model saved.
Loading trained models...
Found 1629 test images.
Test feature matrix shape: (1629, 55)
Predicting test labels...
Test Set Accuracy: 0.14
Total time for entire process: 5156.14 seconds
Image: user001_abjadiyah_035.png, True Label: user001, Predicted: user006
Image: user001_fasayakfeekahum_044.png, True Label: user001, Predicted: user051
Image: user001_fasayakfeekahum_046.png, True Label: user001, Predicted: user002
Image: user001_fasayakfeekahum_050.png, True Label: user001, Predicted: user009
Image: user001_ghaleez_013.png, True Label: user001, Predicted: user044
Image: user001_ghazaal_002.png, True Label: user001, Predicted: user001
Image: user001_ghazaal_007.png, True Label: user001, Predicted: user034
Image: user001_mehras_041.png, True Label: user001, Pred

ORB algorithm with EVALUATION METRICES (ACCURACCY, TIME EFFICIENCY, NUMBER OF KEYPOINTS)

In [10]:
# Define the input and output directories
input_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\AHAWP_splitting_to_train_and_test"
output_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\orbKeypoints_split_new"

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# ORB detector initialization
orb = cv2.ORB_create()

# Function to process images with ORB and save results
def process_with_orb(image_path, output_img_path, output_desc_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Detect keypoints and compute descriptors
    start_time = time.time()
    keypoints, descriptors = orb.detectAndCompute(img, None)
    elapsed_time = time.time() - start_time

    # Draw keypoints on the image
    output_image = cv2.drawKeypoints(img, keypoints, None, color=(255, 0, 255), flags=0)

    # Save the resulting image with keypoints drawn
    cv2.imwrite(output_img_path, output_image)

    # Save the descriptors to a .npy file
    if descriptors is not None:
        output_desc_path = os.path.splitext(output_img_path)[0] + "_desc.npy"
        np.save(output_desc_path, descriptors)  # Save descriptors as .npy files

          # Return the evaluation metrics for this image
    return {
        "num_keypoints": len(keypoints),
        "feature_extraction_time": elapsed_time
    }

    # Metrics collection
metrics = {
    "total_images": 0,
    "total_keypoints": 0,
    "total_time": 0.0,
    "images_processed": []
}

# Process all images in the dataset
for file_path in glob(f"{input_dir}/**/*.png", recursive=True):
    # Create the corresponding output paths
    relative_path = os.path.relpath(file_path, input_dir)
    output_img_path = os.path.join(output_dir, relative_path)
    output_desc_path = os.path.splitext(output_img_path)[0] + "_desc.npy"  # Descriptor file

    # Ensure the output folder exists
    os.makedirs(os.path.dirname(output_img_path), exist_ok=True)

    # Apply ORB and save results
    result =  process_with_orb(file_path, output_img_path, output_desc_path)

 # Update metrics
    metrics["total_images"] += 1
    metrics["total_keypoints"] += result["num_keypoints"]
    metrics["total_time"] += result["feature_extraction_time"]
    metrics["images_processed"].append({
        "file_path": file_path,
        "num_keypoints": result["num_keypoints"],
        "feature_extraction_time": result["feature_extraction_time"]
    })

# Calculate average metrics
average_keypoints = metrics["total_keypoints"] / metrics["total_images"] if metrics["total_images"] > 0 else 0
average_time = metrics["total_time"] / metrics["total_images"] if metrics["total_images"] > 0 else 0

# Print evaluation metrics
print("ORB feature extraction complete for all images.")
print(f"Total Images Processed: {metrics['total_images']}")
print(f"Total Keypoints Detected: {metrics['total_keypoints']}")
print(f"Average Keypoints per Image: {average_keypoints:.2f}")
print(f"Total Feature Extraction Time: {metrics['total_time']:.2f} seconds")
#print(f"Average Feature Extraction Time per Image: {metrics["total_time"] / metrics["total_images"]:.2f} seconds")
#print(f"TOTAL TIME =: {metrics["total_time"]:.2f} seconds")


# Robustness placeholder for further evaluation
#print("Note: Robustness testing for variations in scale, rotation, illumination, and noise requires additional datasets or augmentations.")

ORB feature extraction complete for all images.
Total Images Processed: 8144
Total Keypoints Detected: 468900
Average Keypoints per Image: 57.58
Total Feature Extraction Time: 5.58 seconds


SIFT algorithm with EVALUATION METRICES (ACCURACCY, TIME EFFICIENCY, NUMBER OF KEYPOINTS)

In [11]:
# Path to the directory containing the dataset 
dataset_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\LASTTTTTTTTT FINALLLLLLL ASSIGNMENT2\AHAWP_splitting_to_train_and_test"

# Path to save the output images with SIFT keypoints
output_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\LASTTTTTTTTT FINALLLLLLL ASSIGNMENT2\siftKeypointsWithEvaluation_split"

# Ensure output directory exists
if not os.path.exists(output_path):
    os.makedirs(output_path)

# SIFT detector initialization
sift = cv2.SIFT_create()

# Function to process images with SIFT and save results
def process_with_sift(image_path, output_img_path, output_desc_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Detect SIFT keypoints and descriptors
    start_time = time.time()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    elapsed_time = time.time() - start_time

    # Draw the keypoints on the image
    output_image = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Save the resulting image with keypoints drawn
    cv2.imwrite(output_img_path, output_image)

    # Save the descriptors to a .npy file
    if descriptors is not None:
        output_desc_path = os.path.splitext(output_img_path)[0] + "_descSIFT.npy"
        np.save(output_desc_path, descriptors)  # Save descriptors as .npy files

    # Return the evaluation metrics for this image
    return {
        "num_keypoints": len(keypoints),
        "feature_extraction_time": elapsed_time
    }

# Metrics collection
metrics = {
    "total_images": 0,
    "total_keypoints": 0,
    "total_time": 0.0,
    "images_processed": []
}

# Process all images in the dataset
for file_path in glob(f"{dataset_path}/**/*.png", recursive=True):
    # Create the corresponding output paths
    relative_path = os.path.relpath(file_path, dataset_path)
    output_img_path = os.path.join(output_path, relative_path)
    output_desc_path = os.path.splitext(output_img_path)[0] + "_descSIFT.npy"  # Descriptor file path

    # Ensure the output folder exists
    os.makedirs(os.path.dirname(output_img_path), exist_ok=True)

    # Apply SIFT and save results, collecting metrics
    result = process_with_sift(file_path, output_img_path, output_desc_path)

    # Update metrics
    metrics["total_images"] += 1
    metrics["total_keypoints"] += result["num_keypoints"]
    metrics["total_time"] += result["feature_extraction_time"]
    metrics["images_processed"].append({
        "file_path": file_path,
        "num_keypoints": result["num_keypoints"],
        "feature_extraction_time": result["feature_extraction_time"]
    })

# Calculate average metrics
average_keypoints = metrics["total_keypoints"] / metrics["total_images"] if metrics["total_images"] > 0 else 0
average_time = metrics["total_time"] / metrics["total_images"] if metrics["total_images"] > 0 else 0

# Print evaluation metrics
print("SIFT feature extraction complete for all images.")
print(f"Total Images Processed: {metrics['total_images']}")
print(f"Total Keypoints Detected: {metrics['total_keypoints']}")
print(f"Average Keypoints per Image: {average_keypoints:.2f}")
print(f"Total Feature Extraction Time: {metrics['total_time']:.2f} seconds")
#print(f"Average Feature Extraction Time per Image: {average_time:.2f} seconds")

# Robustness placeholder for further evaluation
#print("Note: Robustness testing for variations in scale, rotation, illumination, and noise requires additional datasets or augmentations.")


SIFT feature extraction complete for all images.
Total Images Processed: 8144
Total Keypoints Detected: 416246
Average Keypoints per Image: 51.11
Total Feature Extraction Time: 19.68 seconds


FINAL ORB WITH SVM WITH TIME EFFICIENCY

In [15]:
import time

# -------------------- CONFIGURATION --------------------
train_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\orbKeypoints_split_new\train"
test_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\orbKeypoints_split_new\test"

k3 = 700

# Generate histogram of visual words
def generate_histogram(descriptors, kmeans_model):
    histogram = np.zeros(kmeans_model.n_clusters)
    clusters = kmeans_model.predict(descriptors)
    for cluster in clusters:
        histogram[cluster] += 1
    return histogram

def extract_descriptors_from_saved(image_paths):
    descriptors_list = []
    labels = []

    for image_path in image_paths:
        # Locate the corresponding descriptor file
        desc_file_path = os.path.splitext(image_path)[0] + "_desc.npy"
        if os.path.exists(desc_file_path):
            descriptors = np.load(desc_file_path)  # Load saved descriptors
            descriptors_list.append(descriptors)

            # Extract label from the folder structure
            label = os.path.basename(os.path.dirname(image_path))
            labels.append(label)
        else:
            print(f"Descriptor file not found for {image_path}")
    return descriptors_list, labels

# -------------------- STEP 1: TRAIN KMEANS (BOVW) --------------------
start_time = time.time()  # Start timing for the entire process

# Get all training image paths
step_start = time.time()
train_image_paths = glob(f"{train_dir}/**/*.png", recursive=True)
print(f"Found {len(train_image_paths)} training images.")
print(f"Time for loading training image paths: {time.time() - step_start:.2f} seconds")

# Extract descriptors and labels
step_start = time.time()
train_descriptors_list, train_labels = extract_descriptors_from_saved(train_image_paths)
all_descriptors = np.vstack(train_descriptors_list)
print(f"Total descriptors shape: {all_descriptors.shape}")
print(f"Time for descriptor extraction: {time.time() - step_start:.2f} seconds")

# Train KMeans
step_start = time.time()
print("Training KMeans to create visual words...")
kmeans = KMeans(n_clusters=k3, random_state=42)
kmeans.fit(all_descriptors)
print(f"Time for KMeans training: {time.time() - step_start:.2f} seconds")

# Generate histograms for training set
step_start = time.time()
X_train_raw = [generate_histogram(des, kmeans) for des in train_descriptors_list]
Y_train = np.array(train_labels)
print(f"Time for histogram generation: {time.time() - step_start:.2f} seconds")

# Compute IDF weights using TfidfTransformer
step_start = time.time()
print("Applying TF-IDF weighting...")
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
X_train_tfidf = tfidf.fit_transform(X_train_raw).toarray()  # TF-IDF weighted histograms
print(f"Time for TF-IDF computation: {time.time() - step_start:.2f} seconds")

# -------------------- STEP 2: TRAIN SVM --------------------
step_start = time.time()
print("Training SVM classifier...")
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=10, gamma=1)
svm.fit(X_train_tfidf, Y_train)  # Use TF-IDF weighted histograms
print(f"Time for SVM training: {time.time() - step_start:.2f} seconds")

# -------------------- STEP 3: EVALUATE ON TEST SET --------------------
# Get all test image paths
step_start = time.time()
test_image_paths = glob(f"{test_dir}/**/*.png", recursive=True)
print(f"Found {len(test_image_paths)} test images.")
print(f"Time for loading test image paths: {time.time() - step_start:.2f} seconds")

# Extract descriptors and labels for the test set
step_start = time.time()
test_descriptors_list, test_labels = extract_descriptors_from_saved(test_image_paths)
print(f"Time for extracting test descriptors: {time.time() - step_start:.2f} seconds")

# Generate histograms for test set
step_start = time.time()
X_test_raw = [generate_histogram(des, kmeans) for des in test_descriptors_list]
print(f"Time for test histogram generation: {time.time() - step_start:.2f} seconds")

# Apply the same TF-IDF transformation to the test data
step_start = time.time()
X_test_tfidf = tfidf.transform(X_test_raw).toarray()
print(f"Time for TF-IDF transformation on test data: {time.time() - step_start:.2f} seconds")

# Predict using the trained SVM
step_start = time.time()
Y_pred = svm.predict(X_test_tfidf)
print(f"Time for SVM prediction: {time.time() - step_start:.2f} seconds")

# Calculate accuracy
step_start = time.time()
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, Y_pred)
print(f"Test Set Accuracy: {accuracy:.2f}")
#print(f"Time for accuracy computation: {time.time() - step_start:.2f} seconds")

print(f"ORB WITH SVM: Total time for entire process: {time.time() - start_time:.2f} seconds")


for i in range(len(test_image_paths)):
    print(f"Image: {os.path.basename(test_image_paths[i])}, True Label: {test_labels[i]}, Predicted: {Y_pred[i]}")




Found 6515 training images.
Time for loading training image paths: 0.06 seconds
Total descriptors shape: (375596, 32)
Time for descriptor extraction: 36.75 seconds
Training KMeans to create visual words...
Time for KMeans training: 68.72 seconds
Time for histogram generation: 14.70 seconds
Applying TF-IDF weighting...
Time for TF-IDF computation: 0.18 seconds
Training SVM classifier...
Time for SVM training: 12.65 seconds
Found 1629 test images.
Time for loading test image paths: 0.02 seconds
Time for extracting test descriptors: 10.32 seconds
Time for test histogram generation: 3.80 seconds
Time for TF-IDF transformation on test data: 0.02 seconds
Time for SVM prediction: 6.19 seconds
Test Set Accuracy: 0.15
Total time for entire process: 153.41 seconds
Image: user001_abjadiyah_035.png, True Label: user001, Predicted: user010
Image: user001_fasayakfeekahum_044.png, True Label: user001, Predicted: user029
Image: user001_fasayakfeekahum_046.png, True Label: user001, Predicted: user010
I

SIFT algorithm with EVALUATION METRICES (ACCURACCY, TIME EFFICIENCY, NUMBER OF KEYPOINTS)

In [16]:
# Path to the directory containing the dataset 
dataset_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\AHAWP_splitting_to_train_and_test"

# Path to save the output images with SIFT keypoints
output_path = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\siftKeypointsWithEvaluation_split"

# Ensure output directory exists
if not os.path.exists(output_path):
    os.makedirs(output_path)

# SIFT detector initialization
sift = cv2.SIFT_create()

# Function to process images with SIFT and save results
def process_with_sift(image_path, output_img_path, output_desc_path):
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Detect SIFT keypoints and descriptors
    start_time = time.time()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    elapsed_time = time.time() - start_time

    # Draw the keypoints on the image
    output_image = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Save the resulting image with keypoints drawn
    cv2.imwrite(output_img_path, output_image)

    # Save the descriptors to a .npy file
    if descriptors is not None:
        output_desc_path = os.path.splitext(output_img_path)[0] + "_descSIFT.npy"
        np.save(output_desc_path, descriptors)  # Save descriptors as .npy files

    # Return the evaluation metrics for this image
    return {
        "num_keypoints": len(keypoints),
        "feature_extraction_time": elapsed_time
    }

# Metrics collection
metrics = {
    "total_images": 0,
    "total_keypoints": 0,
    "total_time": 0.0,
    "images_processed": []
}

# Process all images in the dataset
for file_path in glob(f"{dataset_path}/**/*.png", recursive=True):
    # Create the corresponding output paths
    relative_path = os.path.relpath(file_path, dataset_path)
    output_img_path = os.path.join(output_path, relative_path)
    output_desc_path = os.path.splitext(output_img_path)[0] + "_descSIFT.npy"  # Descriptor file path

    # Ensure the output folder exists
    os.makedirs(os.path.dirname(output_img_path), exist_ok=True)

    # Apply SIFT and save results, collecting metrics
    result = process_with_sift(file_path, output_img_path, output_desc_path)

    # Update metrics
    metrics["total_images"] += 1
    metrics["total_keypoints"] += result["num_keypoints"]
    metrics["total_time"] += result["feature_extraction_time"]
    metrics["images_processed"].append({
        "file_path": file_path,
        "num_keypoints": result["num_keypoints"],
        "feature_extraction_time": result["feature_extraction_time"]
    })

# Calculate average metrics
average_keypoints = metrics["total_keypoints"] / metrics["total_images"] if metrics["total_images"] > 0 else 0
average_time = metrics["total_time"] / metrics["total_images"] if metrics["total_images"] > 0 else 0

# Print evaluation metrics
print("SIFT feature extraction complete for all images.")
print(f"Total Images Processed: {metrics['total_images']}")
print(f"Total Keypoints Detected: {metrics['total_keypoints']}")
print(f"Average Keypoints per Image: {average_keypoints:.2f}")
print(f"Total Feature Extraction Time: {metrics['total_time']:.2f} seconds")
#print(f"Average Feature Extraction Time per Image: {average_time:.2f} seconds")

# Robustness placeholder for further evaluation
#print("Note: Robustness testing for variations in scale, rotation, illumination, and noise requires additional datasets or augmentations.")


SIFT feature extraction complete for all images.
Total Images Processed: 8144
Total Keypoints Detected: 416246
Average Keypoints per Image: 51.11
Total Feature Extraction Time: 21.30 seconds


FINAL SIFT ALGORITHM SVM with time efficiency

In [17]:
import time

# -------------------- CONFIGURATION --------------------
train_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\LASTTTTTTTTT FINALLLLLLL ASSIGNMENT2\siftKeypoints_split\train"
test_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\LASTTTTTTTTT FINALLLLLLL ASSIGNMENT2\siftKeypoints_split\test"

k4 = 700

sift = cv2.SIFT_create()

def extract_descriptors_from_saved(image_paths):
    descriptors_list = []
    labels = []

    for image_path in image_paths:
        # Locate the corresponding descriptor file
        desc_file_path = os.path.splitext(image_path)[0] + "_descSIFT.npy"
        if os.path.exists(desc_file_path):
            descriptors = np.load(desc_file_path)  # Load saved descriptors
            descriptors_list.append(descriptors)

            # Extract label from the folder structure
            label = os.path.basename(os.path.dirname(image_path))
            labels.append(label)
        else:
            print(f"Descriptor file not found for {image_path}")
    return descriptors_list, labels

# -------------------- STEP 1: TRAIN KMEANS (BOVW) --------------------
start_time = time.time()  # Start timing for the entire process

# Get all training image paths
train_image_paths = glob(f"{train_dir}/**/*.png", recursive=True)
print(f"Found {len(train_image_paths)} training images.")

# Extract descriptors and labels
step_start = time.time()  # Start timing for descriptor extraction
train_descriptors_list, train_labels = extract_descriptors_from_saved(train_image_paths)
all_descriptors = np.vstack(train_descriptors_list)
print(f"Total descriptors shape: {all_descriptors.shape}")
print(f"Time for descriptor extraction: {time.time() - step_start:.2f} seconds")

# Train KMeans
step_start = time.time()
print("Training KMeans to create visual words...")
kmeans = KMeans(n_clusters=k4, random_state=42)
kmeans.fit(all_descriptors)
print(f"Time for KMeans training: {time.time() - step_start:.2f} seconds")

# Generate histograms for training set
step_start = time.time()
X_train_raw = [generate_histogram(des, kmeans) for des in train_descriptors_list]
Y_train = np.array(train_labels)
print(f"Time for histogram generation: {time.time() - step_start:.2f} seconds")

# Compute IDF weights using TfidfTransformer
step_start = time.time()
print("Applying TF-IDF weighting...")
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(norm='l2', use_idf=True, smooth_idf=True)
X_train_tfidf = tfidf.fit_transform(X_train_raw).toarray()  # TF-IDF weighted histograms
print(f"Time for TF-IDF computation: {time.time() - step_start:.2f} seconds")

# -------------------- STEP 2: TRAIN SVM --------------------
step_start = time.time()
print("Training SVM classifier...")
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=10, gamma=1)
svm.fit(X_train_tfidf, Y_train)  # Use TF-IDF weighted histograms
print(f"Time for SVM training: {time.time() - step_start:.2f} seconds")

# -------------------- STEP 3: EVALUATE ON TEST SET --------------------
# Get all test image paths
step_start = time.time()
test_image_paths = glob(f"{test_dir}/**/*.png", recursive=True)
print(f"Found {len(test_image_paths)} test images.")
print(f"Time for loading test images: {time.time() - step_start:.2f} seconds")

# Extract descriptors and labels for the test set
step_start = time.time()
test_descriptors_list, test_labels = extract_descriptors_from_saved(test_image_paths)
print(f"Time for extracting test descriptors: {time.time() - step_start:.2f} seconds")

# Generate histograms for test set
step_start = time.time()
X_test_raw = [generate_histogram(des, kmeans) for des in test_descriptors_list]
print(f"Time for test histogram generation: {time.time() - step_start:.2f} seconds")

# Apply the same TF-IDF transformation to the test data
step_start = time.time()
X_test_tfidf = tfidf.transform(X_test_raw).toarray()
print(f"Time for TF-IDF transformation on test data: {time.time() - step_start:.2f} seconds")

# Predict using the trained SVM
step_start = time.time()
Y_pred = svm.predict(X_test_tfidf)
print(f"Time for SVM prediction: {time.time() - step_start:.2f} seconds")

# Calculate accuracy
step_start = time.time()
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, Y_pred)
print(f"Test Set Accuracy: {accuracy:.2f}")
#print(f"Time for accuracy computation: {time.time() - step_start:.2f} seconds")

print(f"Total time for entire process: {time.time() - start_time:.2f} seconds")

for i in range(len(test_image_paths)):
    print(f"Image: {os.path.basename(test_image_paths[i])}, True Label: {test_labels[i]}, Predicted: {Y_pred[i]}")



Found 6515 training images.
Total descriptors shape: (331977, 128)
Time for descriptor extraction: 1.41 seconds
Training KMeans to create visual words...
Time for KMeans training: 183.52 seconds
Time for histogram generation: 15.34 seconds
Applying TF-IDF weighting...
Time for TF-IDF computation: 0.15 seconds
Training SVM classifier...
Time for SVM training: 12.56 seconds
Found 1629 test images.
Time for loading test images: 0.02 seconds
Time for extracting test descriptors: 0.33 seconds
Time for test histogram generation: 3.79 seconds
Time for TF-IDF transformation on test data: 0.02 seconds
Time for SVM prediction: 6.13 seconds
Test Set Accuracy: 0.40
Total time for entire process: 223.35 seconds
Image: user001_abjadiyah_033.png, True Label: user001, Predicted: user054
Image: user001_abjadiyah_034.png, True Label: user001, Predicted: user001
Image: user001_azan_001.png, True Label: user001, Predicted: user001
Image: user001_azan_003.png, True Label: user001, Predicted: user048
Image:

MAKE VARIATIONS ON THE DATASET (SCALE, ROTATION, ILLUMINATION)

In [18]:
# Input dataset path
input_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\AHAWP_splitting_to_train_and_test"
output_dir = r"D:\Jouwana-BZU\4th Year\1st Semester\Computer Vision\1210123_Jouwana\transformed_dataset_final"

# Ensure base output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to apply transformations
def apply_transformations(image, relative_path, transformation_dir):
    output_dir_with_transform = os.path.join(output_dir, transformation_dir, os.path.dirname(relative_path))
    os.makedirs(output_dir_with_transform, exist_ok=True)

    transformations = []

    # Scale variations
    if transformation_dir == "scale":
        scales = [0.5, 0.75, 1.25, 1.5]
        for scale in scales:
            scaled = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
            transformations.append((scaled, f"{os.path.basename(relative_path)}scaled{scale}.png"))

    # Rotation variations
    elif transformation_dir == "rotation":
        angles = [45, 90, 180]
        for angle in angles:
            height, width = image.shape[:2]
            center = (width // 2, height // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(image, rotation_matrix, (width, height))
            transformations.append((rotated, f"{os.path.basename(relative_path)}rotated{angle}.png"))

    # Illumination variations
    elif transformation_dir == "illumination":
        brightness_factors = [0.5, 1.5]
        for factor in brightness_factors:
            brightened = cv2.convertScaleAbs(image, alpha=factor, beta=0)
            transformations.append((brightened, f"{os.path.basename(relative_path)}brightness{factor}.png"))

    # Noise variations
    elif transformation_dir == "noise":
        noise_types = ["gaussian", "salt_and_pepper"]
        for noise_type in noise_types:
            noisy_image = add_noise(image, noise_type)
            transformations.append((noisy_image, f"{os.path.basename(relative_path)}noise{noise_type}.png"))

    # Save all transformed images
    for transformed, output_name in transformations:
        output_path = os.path.join(output_dir_with_transform, output_name)
        cv2.imwrite(output_path, transformed)

# Function to add noise
def add_noise(image, noise_type):
    if noise_type == "gaussian":
        mean, sigma = 0, 10
        gauss = np.random.normal(mean, sigma, image.shape).astype(np.uint8)
        noisy = cv2.add(image, gauss)
        return noisy
    elif noise_type == "salt_and_pepper":
        s_vs_p = 0.5
        amount = 0.02
        noisy = np.copy(image)
        # Salt noise
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
        noisy[coords[0], coords[1]] = 255
        # Pepper noise
        num_pepper = np.ceil(amount * image.size * (1.0 - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
        noisy[coords[0], coords[1]] = 0
        return noisy
    else:
        return image

# Process dataset
splits = ["train", "test"]
transformations = ["scale", "rotation", "illumination", "noise"]

for split in splits:
    split_dir = os.path.join(input_dir, split)
    for file_path in glob(f"{split_dir}/*/.png", recursive=True):
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to load {file_path}")
            continue

        relative_path = os.path.relpath(file_path, input_dir)
        for transformation in transformations:
            apply_transformations(img, relative_path, transformation)

print("All transformations applied and saved in respective directories.")

All transformations applied and saved in respective directories.
